In [18]:
import pandas as pd

In [19]:
df = pd.read_fwf("signal1m_tweets_qrels", names=['topic','iteration','document','rel'])

df[df.document==645936801232912384]
df.shape

(6103, 4)

In [20]:
import tweepy as tw
'''
if need to log in to the twitter account:
username = eugene_tan@mymail.sutd.edu.sg
password = P@ssword1
'''

consumer_key= 'LKvavXktNliE3LL52KNK48oWd'
consumer_secret= 'PsSoEz5JRYdrozOe8uBkgphm3YlTipYREexIyJQi4VDVZv9htI'
access_token= '1411277462630146057-ATYdgNSGb3EGBFim8w1sQ8kRGOR7QH'
access_token_secret= 'S3N1R58XNvrucIOgsKIxZY4CSfngnrd6XHIlWCu0k4PaP'
bearer_token= 'AAAAAAAAAAAAAAAAAAAAAOskRQEAAAAA5JKli8luYWK%2BzRTWfn1kLwjv58s%3DWNBJ8KrY0xEwZBqxqLzCjde2fGWM3wBMeq4uyvWoLX5BNfIHHp'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [15]:
## Scrape Data

def get_tweet(id):
     output = ''
     try:
         output = api.get_status(id=id.document).text
     except:
         print ("failed")
         pass
     return output

print (get_tweet("641855701262966784"))

failed



In [24]:
## Scrape Data
# def get_tweet(id)       output =         t             output = api.get_status(id=id.document).    #     ex                 
#     return tput

# df['text']=df.apply(lambda x:get_tweet(x),xis=1)

df.head()
# df = df[.text!='']
# df.to_csv("dataset_scrapped.csv", header=False)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 11)

In [19]:
data = pd.read_csv('dataset_scrapped.csv', names=['topic','iteration','document','rel', 'text'])
data.head()

,topic,iteration,document,rel,text
0,00ecb565-8dcb-443e-875c-71babdce2269,0,638637447023194112,0,Video: Shippers look to grab a piece of North...
1,00ecb565-8dcb-443e-875c-71babdce2269,0,638727826393444352,1,Police say woman was driving drunk with kids i...
2,00ecb565-8dcb-443e-875c-71babdce2269,0,639043756038074369,1,Hit-and-Run Driver Seriously Injures Man in Po...
3,00ecb565-8dcb-443e-875c-71babdce2269,0,639219133859631104,0,#UPDATE: Man in critical condition after being...
4,00ecb565-8dcb-443e-875c-71babdce2269,0,639325701297041408,1,A Barrie woman that was run over by a drunk dr...


In [49]:
from nltk.corpus import words
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

data = pd.read_csv('dataset_scrapped.csv', names=['topic','iteration','document','rel', 'text'])
data.topic = data.topic.astype(str)
print(data.dtypes)

porter = PorterStemmer() ## stemming reduces the word to root word
words = words.words()
# vsm = {}

# def stem_lower(word):
#     return porter.stem(word.lower())

# for i in list(set(zip(data['document'], data['text']))):
#     # print(i[0], i[1])
#     tokens = word_tokenize(i[1])
#     # print(len(tokens))
#     tokens = list(filter(lambda x: x.isalpha(), tokens))
#     # tokens = list(filter(lambda x: x in words, tokens)) ## this removes too much semantics
#     tokens = list(map(stem_lower, tokens))
#     # print(len(tokens))
#     vsm[str(i[0])] = tokens

def tokenize_stem_lower(text):
    tokens = word_tokenize(text)
    tokens = list(filter(lambda x: x.isalpha(), tokens))
    tokens = [porter.stem(x.lower()) for x in tokens]
    return ' '.join(tokens)

data['clean_text']=data.apply(lambda x: tokenize_stem_lower(x.text), axis=1)

# vsm['645687105964912640']
data.head(10)

topic        object
iteration     int64
document      int64
rel           int64
text         object
dtype: object


,topic,iteration,document,rel,text,clean_text
0,00ecb565-8dcb-443e-875c-71babdce2269,0,638637447023194112,0,Video: Shippers look to grab a piece of North...,video shipper look to grab a piec of northwest...
1,00ecb565-8dcb-443e-875c-71babdce2269,0,638727826393444352,1,Police say woman was driving drunk with kids i...,polic say woman wa drive drunk with kid in the...
2,00ecb565-8dcb-443e-875c-71babdce2269,0,639043756038074369,1,Hit-and-Run Driver Seriously Injures Man in Po...,driver serious injur man in portola district a...
3,00ecb565-8dcb-443e-875c-71babdce2269,0,639219133859631104,0,#UPDATE: Man in critical condition after being...,updat man in critic condit after be shot in th...
4,00ecb565-8dcb-443e-875c-71babdce2269,0,639325701297041408,1,A Barrie woman that was run over by a drunk dr...,a barri woman that wa run over by a drunk driv...
5,00ecb565-8dcb-443e-875c-71babdce2269,0,639362932363653120,0,This photo proves that there are never-ending ...,thi photo prove that there are sight in the no...
6,00ecb565-8dcb-443e-875c-71babdce2269,0,639485269385678849,1,A man was hit by a car while trying to chase a...,a man wa hit by a car while tri to chase after...
7,00ecb565-8dcb-443e-875c-71babdce2269,0,639490286196932608,0,@christopherokey the ruling was that he swung ...,christopherokey the rule wa that he swung and ...
8,00ecb565-8dcb-443e-875c-71babdce2269,0,639535449673850880,0,"""My friend was doing CPR on him. He took a cou...",my friend wa do cpr on him he took a coupl of ...
10,00ecb565-8dcb-443e-875c-71babdce2269,0,639662060368846848,1,RT @Ama_A_Asmah: The women who got out of her ...,rt the women who got out of her car to help ma...


In [115]:
'''
CBOW tries to predict a word on the basis of its neighbors, while Skip Gram tries to predict the neighbors of a word.

In simpler words, CBOW tends to find the probability of a word occurring in a context. So, it generalizes over all the different contexts in which a word can be used. Whereas SkipGram tends to study different contexts separately. Skip Gram needs more data to be trained contains more knowledge about the context.
'''

from sklearn.feature_extraction.text import CountVectorizer

query = 'VIDEO:Good Samaritans Stop Alleged Hit-and-Run Driver in Miami'
query_id = '00ecb565-8dcb-443e-875c-71babdce2269'

vec = CountVectorizer()
# term_doc = vec.fit_transform([query]+list(data.clean_text))
# ind = ['query'] + list(data.document)
# td_df = pd.DataFrame(term_doc.toarray(), columns=vec.get_feature_names(), index=ind)
term_doc = vec.fit_transform(list(data.clean_text))
ind = ['query'] + list(data.document)
td_df = pd.DataFrame(term_doc.toarray(), columns=vec.get_feature_names(), index=list(data.document))

td_df.head(20)

,aaron,abad,abandon,abarrica,abba,abbey,abbeylynnmangu,abbott,abcgrandstand,abdel,...,zippydazipst,zone,zsofimajor,zuckerberg,zuma,zurich,zvishavan,zweigen,öbb,박근혜
638637447023194112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
638727826393444352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639043756038074369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639219133859631104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639325701297041408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639362932363653120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639485269385678849,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639490286196932608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639535449673850880,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639662060368846848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

cos_sim = cosine_similarity(td_df, td_df)
top_10_ind = np.flip(np.argsort(cos_sim[0]))[1:21]
top_10_id = [list(td_df.index)[i] for i in top_10_ind]
print(top_10_ind ,top_10_id)

result = []
for i in top_10_id:
    filt = data[data.document==i]
    for ind, r in filt.iterrows():
        rel = r['rel']
        text = r['text']
        related = r['topic']
        score = 0
        if related==query_id and rel>0:
            score = 1
        result.append({'tweet_id':i, 'text': text, 'related_article':related,'score': score})

results = pd.DataFrame(result)
results


[  53   52   38   40   50 1526   42   22   44 2643 5108   34   46   45
 2732 2150 3878 1470  178 1774] [647491249285525504, 647481801980649472, 645706824293855234, 645789946020806656, 646103787858321408, 638714775137054720, 645900389418557440, 641720036520955904, 645936801232912384, 647455246147526657, 642132786204143620, 644484687637843968, 645979857944477696, 645944922286202881, 643324740682256384, 643704631818891264, 645736657874882560, 642273834809335808, 642120885009149952, 645408290654916608]


,tweet_id,text,related_article,score
0,647491249285525504,Police say they've located the vehicle and dri...,00ecb565-8dcb-443e-875c-71babdce2269,1
1,647481801980649472,#BREAKING: Pedestrian struck in a hit and run ...,00ecb565-8dcb-443e-875c-71babdce2269,1
2,645706824293855234,Police searching for fatal hit &amp; run drive...,00ecb565-8dcb-443e-875c-71babdce2269,1
3,645789946020806656,Innings 6 and 7 for the Mets' pitchers and def...,00ecb565-8dcb-443e-875c-71babdce2269,0
4,646103787858321408,UA student taken to hospital after hit and run...,00ecb565-8dcb-443e-875c-71babdce2269,1
5,638714775137054720,JUST IN: The driver in a crash that killed a t...,4e74bbed-b449-48a8-a71c-f70499105289,0
6,645900389418557440,RT @emmykubainski: A breakthrough in the hunt ...,00ecb565-8dcb-443e-875c-71babdce2269,1
7,641720036520955904,Troopers: Hit and run suspect leads 100+ mph c...,00ecb565-8dcb-443e-875c-71babdce2269,1
8,645936801232912384,Police have located a woman they believe was i...,00ecb565-8dcb-443e-875c-71babdce2269,1
9,647455246147526657,"the #1 and #3 albums in America and the #1, #2...",7d81a98b-8483-4b4c-8c74-55d586f7d863,0


In [109]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean = euclidean_distances(td_df.values[1:], [td_df.values[0]])
top_10_ind = np.argsort(euclidean.T[0])[:20]
top_10_id = [list(td_df.index)[i] for i in top_10_ind]
print(sorted(euclidean[:20]),top_10_ind ,top_10_id)

result = []
for i in top_10_id:
    filt = data[data.document==i]
    for ind, r in filt.iterrows():
        rel = r['rel']
        text = r['text']
        related = r['topic']
        score = 0
        if related==query_id and rel>0:
            score = 1
        result.append({'tweet_id':i, 'text': text, 'related_article':related,'score': score})

results = pd.DataFrame(result)
results

[array([4.12310563]), array([4.12310563]), array([4.24264069]), array([4.35889894]), array([4.35889894]), array([4.58257569]), array([4.69041576]), array([4.69041576]), array([4.79583152]), array([5.]), array([5.]), array([5.]), array([5.29150262]), array([5.29150262]), array([5.38516481]), array([5.47722558]), array([5.83095189]), array([5.91607978]), array([6.8556546]), array([6.8556546])] [2636 2649  434 1104 1573 3695   44 4917 2568  165 3502 3504 4234 1307
 1642  251  441 4263 1546 5244] [646689661369368576, 648126916898213888, 645862396985253888, 647049454030196736, 641612979184623616, 639396730950189056, 645936801232912384, 640809318401581056, 645104094944751616, 647860198770544640, 646391835884171264, 646457573802029056, 642556980868083713, 642618624537178112, 641420495255142400, 644566546153472005, 647655900430344193, 645891976584232960, 642619224083464192, 639120760695558144]


,tweet_id,text,related_article,score
0,646689661369368576,RT @NYCSingleMom: $58 H2O+ GIVEAWAY H2O+ and W...,7d81a98b-8483-4b4c-8c74-55d586f7d863,0
1,648126916898213888,The age of circular views,7d81a98b-8483-4b4c-8c74-55d586f7d863,0
2,645862396985253888,"@ns_brady just said it's arguable, is all",0db352b5-25b0-46fc-84a2-2a9845c9b585,0
3,647049454030196736,Bipartisan clapping in House chamber for Pope'...,32023021-1141-4832-9939-c8442d505b34,0
4,641612979184623616,The Bank of Canada will keep its key interest ...,4fef4b46-17e6-43a9-b762-f68b20c1d4ef,0
5,639396730950189056,Bulgarian Parliament Rejects Anti-Corruption B...,b5a591aa-8dd0-43d2-9961-ab19510f87ac,0
6,645936801232912384,Police have located a woman they believe was i...,00ecb565-8dcb-443e-875c-71babdce2269,1
7,640809318401581056,1000s of famers protest at EU HQ to protest sl...,ef972152-a7fb-4823-9895-62be487d9eaa,0
8,645104094944751616,@rameshfilms Absolutely true. And Steve Jobs t...,7cd39dcf-be13-46a7-a6af-b1d55a42e9e9,0
9,647860198770544640,England score the only try of the first half t...,0429b914-907a-4d79-aed8-5e2a7ebaa230,0


In [116]:
'''from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import gensim

doc1 = TaggedDocument(word_tokenize('My name is eugene'), 'tag1')
doc2 = TaggedDocument(word_tokenize('My name is tan'), 'tag2')
doc3 = TaggedDocument(word_tokenize('My name is alpha'), 'tag3')
sentences = [doc1, doc2, doc3]

model = Doc2Vec(alpha=0.025, min_alpha=0.025)
model.build_vocab(sentences)
# # for epoch in range(10):
model.train(sentences, total_examples=model.corpus_count, epochs=10)
# # model.save('/tmp/my_model.doc2vec')
# q = TaggedDocument('name is e', 'tag4')
# tokens = word_tokenize('name is e')
# query = model.infer_vector(tokens)
# # print(model.docvecs.most_similar(positive=[query], topn=10))
#     # model.alpha -= 0.002  # decrease the learning rate
#     # model.min_alpha = model.alpha
# print(model.dv)'''

## To be tried (using doc2vec)

"from gensim.models import Doc2Vec\nfrom gensim.models.doc2vec import TaggedDocument\nimport gensim\n\ndoc1 = TaggedDocument(word_tokenize('My name is eugene'), 'tag1')\ndoc2 = TaggedDocument(word_tokenize('My name is tan'), 'tag2')\ndoc3 = TaggedDocument(word_tokenize('My name is alpha'), 'tag3')\nsentences = [doc1, doc2, doc3]\n\nmodel = Doc2Vec(alpha=0.025, min_alpha=0.025)\nmodel.build_vocab(sentences)\n# # for epoch in range(10):\nmodel.train(sentences, total_examples=model.corpus_count, epochs=10)\n# # model.save('/tmp/my_model.doc2vec')\n# q = TaggedDocument('name is e', 'tag4')\n# tokens = word_tokenize('name is e')\n# query = model.infer_vector(tokens)\n# # print(model.docvecs.most_similar(positive=[query], topn=10))\n#     # model.alpha -= 0.002  # decrease the learning rate\n#     # model.min_alpha = model.alpha\n# print(model.dv)"